In [ ]:
class Observable:
    
    def __init__(self):
        pass

In [366]:
from typing import List, Callable

class Fluent:
    def __init__(self, **fluents):
        for name, value in fluents.items():
            assert(isinstance(name, str))
            assert(isinstance(value, bool))
            self.name = name
            self.value = value
            break # only the first value is processed
        
    # for if statements
    def __bool__(self):
        return self.value
    
    # for ==, != comparisons
    def __eq__(self, other):
        if isinstance(other, bool):
            return self.value == other
        elif isinstance(other, Fluent):
            return (self.value == other.value) and (self.name == other.name)
        else:
            return False
        
    def release(self):
        self.value != self.value
        
    def positive(self):
        self.value = True
    
    def negative(self):
        self.value = False
    
    def __repr__(self):
        return f"{self.name}={self.value}"
       
class Action:
    
    def __init__(self, name):
        self.name: str = name
        self.causes: Fluent
        self.conditions: List[Fluent] = list()
            
            
            
        
class Model:
    
    def __init__(self):
        
        self.fluents: Dict[str, Fluent] = dict()
        self._causes: Dict[str, List[Callable]] = dict()
    
    def __repr__(self):
        fluents = "\nFLUENTS\n"+"\n".join(map(str, self.fluents.values()))
#         causes =  "\n\nCAUSES\n"+"\n".join(map(str, self._causes.keys()))
        return fluents

    
    def _set(self, fluents: Fluent):
        if isinstance(fluents, List):
            for fluent in fluents:
                self.fluents[fluent.name] = fluent
            return
        
        self.fluents[fluents.name] = fluents
        
    def _check(self, conditions: List[Fluent]) -> bool:
        if conditions is None:
            return True
        
        for f in conditions:
            if f.name not in self.fluents:
                # at least one state not defined
#                 print(f"{f.name} not in {self.fluents.keys()}")
                return False
            
            if f != self.fluents[f.name]:
                # at least one state not satified
#                 print(f"{f} != {self.fluents[f.name]}")
                return False
        return True
    
    def _do_causes(self, method_name):
        for f in self._causes[method_name]:
            f(self)
    
    def _add_causes(self, method_name: str, f: Callable):
        if method_name not in self._causes:
            self._causes[method_name] = []
            setattr(self.__class__, method_name, lambda x: x._do_causes(method_name))
        
        self._causes[method_name].append(f)
    
    def initially(self, **kwargs):
        for key, value in kwargs.items():
            self.fluents[key] = Fluent(**{key:value})
        
            
    def causes(self, action: str, fluents: List[Fluent], conditions: List[Fluent] = None):
        if isinstance(fluents, Fluent):
            fluents = [fluents]
        if isinstance(conditions, Fluent):
            conditions = [conditions]
        
        self._add_causes(action, lambda x: x._set(fluents) if x._check(conditions) else None)
    

In [296]:
f1 = Fluent(key=False)
f2 = Fluent(loaded = False)
f3 = Fluent(key=False)
f4 = Fluent(key=True)

In [133]:
f1 == False

True

In [134]:
f1 == f2

False

In [5]:
f1 == f3

True

In [6]:
f1 == f4

False

In [7]:
f1 == 3

False

In [382]:
m = Model()
m.initially(loaded = False, alive = True)
m.causes("load", Fluent(loaded=True))
m.causes("shoot", Fluent(alive=False), conditions = Fluent(loaded = True))
m.causes("shoot", Fluent(loaded=False))

In [368]:
m


FLUENTS
loaded=False
alive=True

In [369]:
m.shoot()

In [370]:
m


FLUENTS
loaded=False
alive=True

In [372]:
m.load()

In [373]:
m


FLUENTS
loaded=True
alive=True

In [374]:
m.shoot()

In [375]:
m


FLUENTS
loaded=False
alive=False

In [362]:
from copy import deepcopy

class Statement:
    def __init__(self, fluents: List[Fluent], actions: List[str]):
        self.fluents = fluents
        self.actions = actions
        
def call_method(o, name):
    getattr(o, name)()

class Structure:
    
    def __init__(self, model: Model):
        self.model = deepcopy(model)

    def is_statement_true(self, statement: Statement):
        m = deepcopy(self.model)
        
        for action in statement.actions:
            call_method(m, action)
            
        return m._check(statement.fluents)

In [379]:
statement_1 = Statement([Fluent(alive=False)], actions=["shoot"])
statement_2 = Statement([Fluent(alive=False)], actions=["load", "shoot"])

stru = Structure(model=m)

In [380]:
stru.is_statement_true(statement=statement_1)

False

In [381]:
stru.is_statement_true(statement=statement_2)

True